# Generating Objects w/ `Variational Auto Encoder (VAE)`

### Import dependencies

In [1]:
import os
import sys
import datetime as dt

import numpy as np
import tensorflow as tf

from dataset import ImageDataset
%matplotlib inline

### Load dataset

In [ ]:
data_dir = 'datasets/101_ObjectCategories/'
save_file = 'saved/'

In [ ]:
data = ImageDataset(data_dir=data_dir, size=64, grayscale=True, flatten=True)
data.create()